In [1]:
from mylib import preprocess
import pandas as pd
import os

pretest = pd.read_csv(os.path.join('data', 'test_users.csv'), header=0, parse_dates=[1,2,3])
pretrain = pd.read_csv(os.path.join('data', 'train_users_2.csv'), header=0, parse_dates=[1,2,3])
df_sessions = pd.read_csv("data\sessions.csv", encoding='utf8')

In [2]:
train, test, y, le = preprocess.make_user_features(pretrain, pretest)

In [3]:
data = pd.concat((train, test), axis=0, ignore_index=True)
final = preprocess.make_sessions_features(data, df_sessions)

In [4]:
final[:train.shape[0]].shape

(213451, 549)

In [5]:
from sklearn.cross_validation import train_test_split
our_train = final[:train.shape[0]]
X_train, X_test, y_train, y_test = train_test_split(our_train, y, test_size=0.33, random_state=42, stratify=y)

In [6]:
X_train.shape

(143012, 549)

In [7]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
clf = RandomForestClassifier(n_estimators=160, oob_score=True, n_jobs=-1, criterion='entropy')
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=160, n_jobs=-1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [8]:
import numpy as np
np.count_nonzero(clf.feature_importances_ < 1e-4)

313

In [9]:
unimportant_features = clf.feature_importances_ < 1e-4
clean_X_train = X_train.ix[:, ~unimportant_features]

In [11]:
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import BaggingClassifier
from pprint import pprint
bagg = BaggingClassifier(random_state=42)

param_grid = {"n_estimators": [10, 50, 100],
              "max_samples": [0.1, 0.5, 1.0],
              "max_features": [0.1, 0.5, 1.0]}

search = GridSearchCV(bagg, param_grid, scoring='f1_weighted', cv=3, verbose=4, n_jobs=2)
search.fit(clean_X_train, y_train)
pprint(sorted(search.grid_scores_, key=lambda x: -x.mean_validation_score))

[Parallel(n_jobs=2)]: Done  21 tasks      | elapsed:  2.9min
[Parallel(n_jobs=2)]: Done  81 out of  81 | elapsed: 103.4min finished


Fitting 3 folds for each of 27 candidates, totalling 81 fits
[mean: 0.59378, std: 0.00149, params: {'max_features': 1.0, 'max_samples': 0.1, 'n_estimators': 100},
 mean: 0.59077, std: 0.00205, params: {'max_features': 1.0, 'max_samples': 0.1, 'n_estimators': 50},
 mean: 0.58520, std: 0.00156, params: {'max_features': 0.5, 'max_samples': 0.1, 'n_estimators': 100},
 mean: 0.58505, std: 0.00087, params: {'max_features': 1.0, 'max_samples': 0.5, 'n_estimators': 100},
 mean: 0.58318, std: 0.00039, params: {'max_features': 0.5, 'max_samples': 0.5, 'n_estimators': 100},
 mean: 0.58218, std: 0.00003, params: {'max_features': 1.0, 'max_samples': 0.5, 'n_estimators': 50},
 mean: 0.58107, std: 0.00090, params: {'max_features': 0.5, 'max_samples': 0.1, 'n_estimators': 50},
 mean: 0.58057, std: 0.00058, params: {'max_features': 0.5, 'max_samples': 1.0, 'n_estimators': 100},
 mean: 0.58056, std: 0.00060, params: {'max_features': 0.5, 'max_samples': 0.5, 'n_estimators': 50},
 mean: 0.57927, std: 0.00